In [1]:
import numpy as np

In [2]:
M = 5
N = 5
L = np.zeros((int(M), int(N)), dtype='float64')
U = np.zeros((int(M), int(N)), dtype='float64')
ones = np.ones((int(N)))
U = np.diag(ones)

L = U
print(L)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [14]:
def decompose_lu(A):
    N = A.shape[0]
    ones = np.ones(A.shape[0])
    U = np.diag(ones)
    L = U
    
    for j in range(1, N):
        U[0][j] = A[1][j]/L[1][1]
        L[j][0] = A[j][1]/U[1][1]
    #print(U)
    for i in range(1, N-1):
        for j in range(1, N):
            aux = 0
            for k in range(i-1):
                aux += L[i][k]*U[k][i]
            U[i][j] = (1/L[i][i])/(A[i][j] - aux)
            
            for k in range(i-1):
                aux += L[k][i]*U[i][k]
            L[j][i] = (1/U[i][i])/(A[j][i] - aux)
    return L, U

In [15]:
l, u = decompose_lu(A)
print(l)

[[2. 0. 0. 0. 0.]
 [2. 6. 0. 0. 0.]
 [6. 4. 8. 0. 0.]
 [4. 7. 2. 5. 0.]
 [3. 6. 2. 4. 6.]]
[[2. 2. 3. 4. 0.]
 [0. 6. 5. 6. 3.]
 [0. 0. 8. 2. 4.]
 [0. 0. 0. 5. 3.]
 [0. 0. 0. 0. 9.]]
[[  4.   4.   6.   8.   0.]
 [  4.  40.  36.  44.  18.]
 [ 12.  36. 102.  64.  44.]
 [  8.  50.  63.  87.  44.]
 [  6.  42.  55.  72.  92.]]
[[ 1.00000000e+00  4.00000000e+01  3.60000000e+01  4.40000000e+01
   1.80000000e+01]
 [ 4.00000000e+01  1.00000000e+00 -3.91236307e-04 -2.29568388e-04
   5.55555556e-02]
 [ 3.60000000e+01 -7.93650794e-04  4.79518072e-01 -2.30627306e-04
  -1.66567663e-03]
 [ 5.00000000e+01 -4.65116181e-04 -4.67945718e-04  4.89914213e-01
  -9.46741042e-04]
 [ 4.20000000e+01  2.38095238e-02 -8.22005178e-04 -4.71620538e-04
   1.00000000e+00]]


In [25]:
def cholesky(A):
    L = np.zeros_like(A)
    n = len(L)
    for i in range(n):
        for j in range(i+1):
            if i==j:
                val = A[i,i] - np.sum(np.square(L[i,:i]))
                if val<0:
                    return 0.0
                L[i,i] = np.sqrt(val)
            else:
                L[i,j] = (A[i,j] - np.sum(L[i,:j]*L[j,:j]))/L[j,j]
#     for i in range(L.shape[0]):
#         for j in range(L.shape[1]):
#             L[i, j] /= D[i, i]
        
    return L

In [27]:
Lp = np.array(([[2, 0, 0, 0, 0], [2, 6, 0, 0, 0], [6, 4, 8, 0, 0], [4, 7, 2 , 5, 0], [3, 6, 2, 4, 6]]), dtype="float64")
Up = np.array(([[2, 2, 3, 4, 0], [0, 6, 5, 6, 3], [0, 0, 8, 2, 4], [0, 0, 0, 5, 3], [0, 0, 0, 0, 9]]), dtype="float64")


print(Lp)
print(Up)
A = np.dot(Lp, Up)
print(A)
L = cholesky(A)


[[2. 0. 0. 0. 0.]
 [2. 6. 0. 0. 0.]
 [6. 4. 8. 0. 0.]
 [4. 7. 2. 5. 0.]
 [3. 6. 2. 4. 6.]]
[[2. 2. 3. 4. 0.]
 [0. 6. 5. 6. 3.]
 [0. 0. 8. 2. 4.]
 [0. 0. 0. 5. 3.]
 [0. 0. 0. 0. 9.]]
[[  4.   4.   6.   8.   0.]
 [  4.  40.  36.  44.  18.]
 [ 12.  36. 102.  64.  44.]
 [  8.  50.  63.  87.  44.]
 [  6.  42.  55.  72.  92.]]


In [42]:
def LUDecompose(table):
    # Table that contains our data
    # Table has to be a square array so we need to check first
    rows, columns = np.shape(table)
    L = np.zeros((rows, columns))
    U = np.zeros((rows, columns))
    if rows != columns:
        return []
    for i in range(columns):
        for j in range(i - 1):
            sum = 0
            for k in range(j - 1):
                sum += L[i][k] * U[k][j]
            L[i][j] = (table[i][j] - sum) / U[j][j]
        L[i][i] = 1
        for j in range(i - 1, columns):
            sum1 = 0
            for k in range(i - 1):
                sum1 += L[i][k] * U[k][j]
            U[i][j] = table[i][j] - sum1
    return L, U
L, U = LUDecompose(A)
print(L)
print(U)

[[1.         0.         0.         0.         0.        ]
 [0.         1.         0.         0.         0.        ]
 [3.         0.         1.         0.         0.        ]
 [2.         1.25       0.         1.         0.        ]
 [1.5        1.05       0.54761905 0.         1.        ]]
[[ 4.         4.         6.         8.         0.       ]
 [ 4.        40.        36.        44.        18.       ]
 [ 0.        24.        84.        40.        44.       ]
 [ 0.         0.         6.        16.        21.5      ]
 [ 0.         0.         0.        -8.1047619 49.0047619]]


In [50]:
def gera_matriz(ł, N):
    A = np.zeros((N, N))
    for i in range(N):
        A[i, i] = 1 + 2*ł
    for i in range(1, N):
        A[i, i-1] = -ł
    for i in range(0, N-1):
        A[i, i+1] = -ł
    return A

In [54]:
lamb = 0.5
N = 3
A = gera_matriz(lamb, N)
L, U = LUDecompose(A)
print(L)
print(U)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[ 2.  -0.5  0. ]
 [-0.5  2.  -0.5]
 [ 0.  -0.5  2. ]]


In [29]:
aux = np.dot(L, D)
np.dot(aux, np.transpose(L))

array([[  16.        ,   16.        ,   48.        ,   32.        ,
          24.        ],
       [  16.        , 1456.        , 1008.        , 1712.        ,
        1464.        ],
       [  48.        , 1008.        , 5884.        , 2338.        ,
        2358.        ],
       [  32.        , 1712.        , 2338.        , 3974.3       ,
        3336.9       ],
       [  24.        , 1464.        , 2358.        , 3336.9       ,
        5775.26588355]])

In [9]:
np.dot(L, np.transpose(L))

array([[0.25      , 0.025     , 0.02941176, 0.02298851, 0.01630435],
       [0.025     , 0.025     , 0.00882353, 0.01436782, 0.01141304],
       [0.02941176, 0.00882353, 0.00980392, 0.00709939, 0.00586104],
       [0.02298851, 0.01436782, 0.00709939, 0.01149425, 0.0089955 ],
       [0.01630435, 0.01141304, 0.00586104, 0.0089955 , 0.01086957]],
      dtype=float32)